In [7]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import random
from torch import nn

# 1、生成数据集

其中`features`是训练数据特征，`labels`是标签

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


In [3]:
features

tensor([[ 0.5352,  0.1460],
        [-0.4234, -0.5739],
        [-1.1713, -1.9522],
        ...,
        [-0.0642, -0.5281],
        [-0.3597, -2.8097],
        [-1.3384,  0.1425]])

# 2、读取数据

`PyTorch`提供了`data`包来读取数据。由于`data`常用作变量名，我们将导入的`data`模块用`Data`代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [5]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合起来
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

让我们读取并打印第一个小批量数据样本看看：

In [6]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.0296,  0.7181],
        [ 0.6469, -1.2057],
        [-0.1537,  1.7291],
        [ 0.0691, -1.2135],
        [-0.7072, -1.1808],
        [ 0.1855, -0.7341],
        [ 0.0417,  0.8716],
        [ 1.1683, -0.3736],
        [-0.1389, -1.0887],
        [-1.2679, -0.9049]]) tensor([ 1.7980,  9.5870, -1.9948,  8.4590,  6.8163,  7.0547,  1.3218,  7.7953,
         7.6382,  4.7498])


# 3、定义模型

In [9]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        """定义前向传播"""
        y = self.linear(x)
        return y

In [10]:
net = LinearNet(num_inputs)
net

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

事实上我们还可以用`nn.Sequential`来更加方便地搭建网络，`Sequential`是一个有序的容器，网络层将按照在传入`Sequential`的顺序依次被添加到计算图中。

In [11]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [12]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
net

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [13]:
# 写法三
from collections import OrderedDict
d = OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
])
net = nn.Sequential(d)
net

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

可以通过`net.parameters()`来查看模型所有的可学习参数，此函数将返回一个生成器

In [14]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.6512, -0.5666]], requires_grad=True)
Parameter containing:
tensor([0.2454], requires_grad=True)


作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。

注意：`torch.nn`仅支持输入一个batch的样本不支持单个样本输入，如果只有单个样本，可使用`input.unsqueeze(0)`来添加一维

# 4、初始化模型参数
在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。PyTorch在`init`模块中提供了多种参数初始化方法。这里的`init`是`initializer`的缩写形式。我们通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零

In [15]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)
# 也可以直接修改bias的data
# net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

# 5、定义损失函数
PyTorch在`nn`模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为`nn.Module`的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [16]:
loss = nn.MSELoss()

# 6、定义优化算法
`torch.optim`模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化`net`所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [17]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)

我们还可以为不同子网络设置不同的学习率，这在`finetune`时经常用到。例：

```python
optimizer2 = optim.SGD([
    # 如果对某个参数不指定学习率，就使用最外层的默认学习率
    {
        'params': net.subnet1.parameters()
    },
    {
        'params': net.subnet2.parameters(),
        'lr': 0.01
    }
 
], lr=0.03)
```

有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。一种是修改`optimizer.param_groups`中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

In [19]:
# 调整学习率为之前的0.1倍
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

# 7、训练模型
我们通过调用`optim`实例的`step`函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在`step`函数中指明批量大小，从而对批量中样本梯度求平均。

In [21]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        # 梯度清零，等价于net.zero_grad()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 3.075946
epoch 2, loss: 1.328295
epoch 3, loss: 1.041149


下面我们分别比较学到的模型参数和真实的模型参数。我们从`net`获得需要的层，并访问其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [22]:
dense = net[0]
print(dense.weight)
print(dense.bias)

Parameter containing:
tensor([[ 1.5751, -2.9107]], requires_grad=True)
Parameter containing:
tensor([3.4801], requires_grad=True)


# 8、小结
- `torch.utils.data`模块提供了有关数据处理的工具
- `torch.nn`模块定义了大量神经网络的层
- `torch.nn.init`模块定义了各种初始化方法
- `torch.optim`模块提供了很多常用的优化算法